In [45]:
import pandas as pd
import csv

In [2]:
file = pd.read_excel("Copy of Trade History.xlsx")
starting_point = pd.read_excel("Portfolio at end of 2009.xlsx")
file.head()
starting_point.head()
print(starting_point)



    Unnamed: 0 Trade Date Sector Ticker            Security Description  \
0          NaN 2009-12-31     CD    DIS                  WALT DISNEY CO   
1          NaN 2009-12-31     CD    MCD                 MCDONALD'S CORP   
2          NaN 2009-12-31     CD    AZO                    AUTOZONE INC   
3          NaN 2009-12-31     CD    VFC                         VF CORP   
4          NaN 2009-12-31     CD    LOW            LOWE'S COMPANIES INC   
5          NaN 2009-12-31     CS     KO           THE COCA-COLA COMPANY   
6          NaN 2009-12-31     CS     PM  PHILIP MORRIS INTERNATIONAL IN   
7          NaN 2009-12-31     CS    CVS                 CVS HEALTH CORP   
8          NaN 2009-12-31     CS    RAH            RALCORP HOLDINGS INC   
9          NaN 2009-12-31     CS    DEO                      DIAGEO PLC   
10         NaN 2009-12-31     CS     PG    THE PROCTER & GAMBLE COMPANY   
11         NaN 2009-12-31      E    APA                     APACHE CORP   
12         NaN 2009-12-31

In [3]:
#Makes the starting point portfolio which will be inputed below in make_portfolio
holdings = {}
for row in starting_point.itertuples():
    holdings[row[4]] = row[7]
print(holdings)    

#Fixing for now 
holdings.update({'FTR':"*"})
print(len(holdings))


{'DIS': 2650, 'MCD': 1275, 'AZO': 290, 'VFC': 632, 'LOW': 2500, 'KO': 1400, 'PM': 1550, 'CVS': 2750, 'RAH': 950, 'DEO': 1200, 'PG': 1600, 'APA': 1140, 'XOM': 1645, 'NE': 3990, 'SLB': 1030, 'CHK': 2925, 'SWN': 1538, 'HCBK': 3830, 'GS': 625, 'BLK': 285, 'MET': 1340, 'IVZ': 2230, 'JPM': 3308, 'CS': 1110, 'SIVB': 1200, 'GILD': 2185, 'ABT': 2050, 'TEVA': 2290, 'JNJ': 1050, 'HAE': 1500, 'WM': 1725, 'BUCY': 1850, 'NOC': 1250, 'UNP': 1225, 'HON': 2775, 'IBM': 1020, 'GOOGL': 460, 'CERN': 1560, 'APH': 2020, 'MANT': 1460, 'SNPS': 3295, 'SY': 2240, 'FCX': 538, 'BHP': 525, 'PPG': 1320, 'PX': 210, 'GEF': 672, 'CSCO': 1294, 'AMT': 800, 'VZ': 1980, 'OGE': 1000, 'NEE': 700, 'WEC': 1620}
54


In [4]:
#Simple EDA
file.describe()
file.head()
file.tail()
file.dtypes

Unnamed: 0                     float64
Trade Date              datetime64[ns]
Sector                          object
Ticker                          object
Security Description            object
Total                          float64
Shares                           int64
Price                          float64
Comm                           float64
Fees                           float64
Settle Date             datetime64[ns]
Principal                      float64
Trade Type                      object
Unnamed: 13                     object
Unnamed: 14                     object
dtype: object

In [5]:
cleaned = file[['Trade Date','Sector','Ticker','Security Description','Total','Shares','Price','Comm','Fees','Principal','Trade Type']].copy()
cleaned.columns

Index(['Trade Date', 'Sector', 'Ticker', 'Security Description', 'Total',
       'Shares', 'Price', 'Comm', 'Fees', 'Principal', 'Trade Type'],
      dtype='object')

In [6]:
cleaned.columns = ['trade_date','Sector', 'Ticker', 'Security Description', 'Total', 'Shares', 'Price','Comm', 'Fees', 'Principal', 'Trade Type']
cleaned['Shares']= cleaned['Shares'].astype(int)
cleaned.head(15)
cleaned.dtypes

trade_date              datetime64[ns]
Sector                          object
Ticker                          object
Security Description            object
Total                          float64
Shares                           int64
Price                          float64
Comm                           float64
Fees                           float64
Principal                      float64
Trade Type                      object
dtype: object

In [43]:
def make_portfolio(datatable,date):
    ts = pd.to_datetime(date)
    
    # Make the initial Portfoliol loaded in an an excel file name "Starting Point" above
    holdings = {}
    for row in starting_point.itertuples():
        holdings[row[4]] = row[7]   
        #Fixing for now 
    holdings.update({'FTR':"*"})
    
    #Create a copy of the trade history up to the provided date and execute transactions
    
    working = datatable[datatable.trade_date <= ts]

    for row in working.itertuples():
        type_change = row[11]
        shares = int(row[6])
        ticker = row[3]
        current_date = row[1]


        if type_change == "ADD":
            holdings[ticker] = holdings.get(ticker, "Error") + shares 

        elif type_change == "SHAVE":
            holdings[ticker] = holdings.get(ticker,"Error") + shares


        elif type_change == "BUY":
            holdings.update({ticker:shares})

        elif type_change == "SELL":
            holdings.pop(ticker)

        else:
            print("error")

    

    print(len(holdings))
    return(holdings)

In [95]:
 a = make_portfolio(cleaned,'3/10/2012')

57


In [110]:
use = list(a.items())
df=pd.DataFrame(use,columns = ['Ticker','Shares'])
df['Ticker'] = df['Ticker'].astype(str)
df.sort_values(by='Ticker')
df.head(50)

,Ticker,Shares
0,MCD,540
1,PM,1110
2,PG,1330
3,IBM,555
4,GOOGL,460
5,PPG,1320
6,AMT,800
7,VZ,1135
8,WEC,1620
9,ANTM,875


,Ticker,Shares
0,MCD,540
1,PM,1110
2,PG,1330
3,IBM,555
4,GOOGL,460
